# Загрузка библиотек

In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import numpy as np
from torch import nn
from torchvision import datasets, transforms, models
from collections import OrderedDict
from PIL import Image
import PIL
from torch.optim import lr_scheduler
import os
from sklearn.model_selection import train_test_split
import shutil
import gc
from tqdm import tqdm
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader, Dataset
from IPython.display import clear_output

In [11]:
RADNDOM_STATE = 42
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Формирование датасета

In [24]:
df = pd.read_pickle('../../data/common_files/df_markup.pkl')
df['file_name'] = df['file_name'].apply(
    lambda x: "..\\" + x
)
df = df[['file_name', 'quality_photo']]
df.head()

,file_name,quality_photo
0,..\..\data\sorted_data_merged\damaged\1.jpg,1
1,..\..\data\sorted_data_merged\damaged\2.jpg,1
2,..\..\data\sorted_data_merged\damaged\3.jpg,1
3,..\..\data\sorted_data_merged\damaged\4.jpg,1
4,..\..\data\sorted_data_merged\damaged\5.jpg,1


In [32]:
train, test = train_test_split(df, stratify=df['quality_photo'], random_state=RADNDOM_STATE)

In [40]:
train_dir = 'data/train'
test_dir = 'data/test'

In [37]:
if not os.path.exists('data'):
    os.makedirs('data')
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
        if not os.path.exists('data/train/quality_positive'):
            os.makedirs('data/train/quality_positive')
        if not os.path.exists('data/train/quality_negative'):
            os.makedirs('data/train/quality_negative')
            
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)
        if not os.path.exists('data/test/quality_positive'):
            os.makedirs('data/test/quality_positive')
        if not os.path.exists('data/test/quality_negative'):
            os.makedirs('data/test/quality_negative')

In [38]:
train_positive = train[train['quality_photo']==1]
train_negative = train[train['quality_photo']==0]

for file in tqdm(train_positive['file_name']):
    shutil.copy(file, 'data/train/quality_positive')
for file in tqdm(train_negative['file_name']):
    shutil.copy(file, 'data/train/quality_negative')

test_positive = test[test['quality_photo']==1]
test_negative = test[test['quality_photo']==0]

for file in tqdm(test_positive['file_name']):
    shutil.copy(file, 'data/test/quality_positive')
for file in tqdm(test_negative['file_name']):
    shutil.copy(file, 'data/test/quality_negative')

100%|██████████| 139/139 [00:00<00:00, 1775.52it/s]


In [41]:
train_transforms = transforms.Compose([
        transforms.RandomRotation(30),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
test_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [42]:
train_data = datasets.ImageFolder(root=train_dir, # target folder of images
                                  transform=train_transforms, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)
test_data = datasets.ImageFolder(root=test_dir, 
                                 transform=test_transforms)
print(f"Train data:\n{train_data}\nTest data:\n{test_data}")

Train data:
Dataset ImageFolder
    Number of datapoints: 6383
    Root location: data/train
    StandardTransform
Transform: Compose(
               RandomRotation(degrees=[-30.0, 30.0], interpolation=nearest, expand=False, fill=0)
               RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear), antialias=None)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 2128
    Root location: data/test
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


In [43]:
# Can also get class names as a dict
class_dict = train_data.class_to_idx
class_dict

{'quality_negative': 0, 'quality_positive': 1}

In [45]:
# Setup batch size and number of workers 
BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()
print(f"Creating DataLoader's with batch size {BATCH_SIZE} and {NUM_WORKERS} workers.")

# Create DataLoader's
train_dataloader = DataLoader(train_data, 
                                     batch_size=BATCH_SIZE, 
                                     shuffle=True, 
                                     num_workers=NUM_WORKERS)

test_dataloader = DataLoader(test_data, 
                                    batch_size=BATCH_SIZE, 
                                    shuffle=False, 
                                    num_workers=NUM_WORKERS)

train_dataloader, test_dataloader

Creating DataLoader's with batch size 32 and 20 workers.


(<torch.utils.data.dataloader.DataLoader at 0x22668a302e0>,
 <torch.utils.data.dataloader.DataLoader at 0x22668a30070>)

# Fine-tuning ResNet 50

In [46]:
sns.set_style('whitegrid')
plt.rcParams.update({'font.size': 15})


def plot_losses(train_losses, test_losses, train_accuracies, test_accuracies, train_f1s, test_f1s):
    clear_output()
    fig, axs = plt.subplots(1, 3, figsize=(20, 10))
    plt.rcParams['font.size'] = '12'
    axs[0].plot(range(1, len(train_losses) + 1), train_losses, label='train')
    axs[0].plot(range(1, len(test_losses) + 1), test_losses, label='test')
    axs[0].set_ylabel('loss')

    axs[1].plot(range(1, len(train_accuracies) + 1), train_accuracies, label='train')
    axs[1].plot(range(1, len(test_accuracies) + 1), test_accuracies, label='test')
    axs[1].set_ylabel('accuracy')

    axs[2].plot(range(1, len(train_f1s) + 1), train_f1s, label='train')
    axs[2].plot(range(1, len(test_f1s) + 1), test_f1s, label='test')
    axs[2].set_ylabel('F1')

    for ax in axs:
        ax.set_xlabel('epoch')
        ax.legend()

    plt.show()

In [55]:
def training_epoch(model, optimizer, criterion, train_loader, tqdm_desc):
    train_loss, train_accuracy, f1_score_value = 0.0, 0.0, 0.0
    model.train()
    for images, labels in tqdm(train_loader, desc=tqdm_desc):
        images = images.to(device)  # images: batch_size x num_channels x height x width
        labels = labels.to(device)  # labels: batch_size

        optimizer.zero_grad()
        logits = model(images)  # logits: batch_size x num_classes
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.shape[0]
        train_accuracy += (logits.argmax(dim=1) == labels).sum().item()

        # Calculate F1-score
        predictions = logits.argmax(dim=1)
        f1_score_value += f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average='macro')
    
    train_loss /= len(train_loader.dataset)
    train_accuracy /= len(train_loader.dataset)
    f1_score_value /= len(train_loader)
    
    return train_loss, train_accuracy, f1_score_value


@torch.no_grad()
def validation_epoch(model, criterion, test_loader, tqdm_desc):
    test_loss, test_accuracy, test_f1 = 0.0, 0.0, 0.0
    model.eval()
    for images, labels in tqdm(test_loader, desc=tqdm_desc):
        images = images.to(device)  # images: batch_size x num_channels x height x width
        labels = labels.to(device)  # labels: batch_size
        logits = model(images)  # logits: batch_size x num_classes
        loss = criterion(logits, labels)

        test_loss += loss.item() * images.shape[0]
        test_accuracy += (logits.argmax(dim=1) == labels).sum().item()
        # Calculate F1-score
        predictions = logits.argmax(dim=1)
        test_f1 += f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average='macro')

    test_loss /= len(test_loader.dataset)
    test_accuracy /= len(test_loader.dataset)
    test_f1 /= len(test_loader)

    return test_loss, test_accuracy, test_f1

    
def train(model, optimizer, scheduler, criterion, train_loader, test_loader, num_epochs):
    train_losses, train_accuracies, train_f1s = [], [], []
    test_losses, test_accuracies, test_f1s = [], [], []

    for epoch in range(1, num_epochs + 1):
        train_loss, train_accuracy, train_f1 = training_epoch(
            model, optimizer, criterion, train_loader,
            tqdm_desc=f'Training {epoch}/{num_epochs}'
        )
        test_loss, test_accuracy, test_f1 = validation_epoch(
            model, criterion, test_loader,
            tqdm_desc=f'Validating {epoch}/{num_epochs}'
        )

        if scheduler is not None:
            scheduler.step()

        train_losses += [train_loss]
        train_accuracies += [train_accuracy]
        train_f1s += [train_f1]
        test_losses += [test_loss]
        test_accuracies += [test_accuracy]
        test_f1s += [test_f1]
        plot_losses(train_losses, test_losses, train_accuracies, test_accuracies, train_f1s, test_f1s)

    return train_losses, test_losses, train_accuracies, test_accuracies, train_f1s, test_f1s

In [56]:
model = models.resnet152()
classifier = nn.Sequential(OrderedDict([
                        ('fc1', nn.Linear(2048, 1024)),
                        ('relu', nn.ReLU()),
                        ('fc2', nn.Linear(1024, 2)),
                        ('output', nn.LogSoftmax(dim=1))
                        ]))

# Replacing the pretrained model classifier with our classifier
model.fc = classifier
model = model.to(device)

In [57]:
num_epochs = 10

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epochs)

In [61]:
gc.collect()
train_losses, test_losses, train_accuracies, test_accuracies, train_f1s, test_f1s = train(
    model, optimizer, scheduler, criterion, train_dataloader, test_dataloader, num_epochs
)

Training 1/10:  36%|███▋      | 73/200 [23:38<41:07, 19.43s/it]  


KeyboardInterrupt: 

Не сработало :(

![Alt text](image.png)